In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

2025/08/31 13:42:06 INFO mlflow.tracking.fluent: Experiment with name 'rlvr-eval' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [3]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-7B-Instruct']
Using model: Qwen/Qwen2.5-7B-Instruct


In [4]:
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-08-31 13:42:10 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 100 rollouts: 100%|██████████| 100/100 [00:00<00:00, 923.02it/s]


[Trace(trace_id=tr-2700080ae28e598ce7c9adbd7b3849cc), Trace(trace_id=tr-4bdbede8a940ce3faf26af35c91780cb), Trace(trace_id=tr-ef676be570e4a8e1c7b74a4410b1a355), Trace(trace_id=tr-52033eb099b1597309fc4adbcacc9e37), Trace(trace_id=tr-3f0edfe8a4348ef828000e86a384a4f2), Trace(trace_id=tr-e2f52765b87e302418bcd91ec989dfdd), Trace(trace_id=tr-24d1132368fa31970afe74a26825fb14), Trace(trace_id=tr-c70fc3572ea5da446349ada78309d616), Trace(trace_id=tr-8616ad413d40d39b008dd52923428dff), Trace(trace_id=tr-64ee87c0a27fa45342dcfb03e5c8aea3)]

In [5]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [6]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'role': 'system', 'content': "Answer the question based on the information provided by tools.\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve documents\n3. Continue until you find the answer through multi-hop reasoning. The question is answerable from the docs. \n4. In the **last** step:\n    - Reflect on your previous steps inside <think> tags\n    - Cite the documents you base your answer on inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n    - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking and explanation here]\n</think> \n<cite>\n[IDs of the documents that back your answer]\n</cite>\n<answer>\n[your final answer in **a few words**. no explanation here.]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Make one tool call per step.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue searching un

/tmp/ipykernel_254913/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [7]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 100
})

In [8]:
ds.to_json("../outputs/musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2733461

In [9]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,[{'content': '<think> To find out what gun was...,LeMat revolver,default,0.245000,"{'answers': ['lemat revolver', 'LeMat Revolver...",0.0,0.0,0.5,0.2,0.5,1.000000,0.245000
1,[{'content': 'Answer the question based on the...,[{'content': '<think> To find the Vice-Preside...,Vitor Constâncio,default,0.000000,"{'answers': ['Vitor Constâncio', 'vitor constâ...",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,[{'content': 'Answer the question based on the...,[{'content': '<think> To find the distance fro...,45,default,0.008333,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",0.0,0.0,0.0,0.0,0.0,0.333333,0.008333
3,[{'content': 'Answer the question based on the...,[{'content': '<think> The question asks about ...,three different relationships he had in the past,default,0.350000,{'answers': ['three different relationships he...,0.0,0.0,1.0,1.0,0.0,0.000000,0.350000
4,[{'content': 'Answer the question based on the...,[{'content': '<think> To find the notable pop ...,Michael Bublé,default,0.175000,"{'answers': ['Michael Bublé', 'michael bublé']...",0.0,0.0,0.5,0.5,0.0,0.000000,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[{'content': 'Answer the question based on the...,"[{'content': '<think> First, I need to find ou...",two alliances,default,0.000000,"{'answers': ['two alliances'], 'docs': [{'body...",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
96,[{'content': 'Answer the question based on the...,[{'content': '<think> The question asks about ...,John Kukuzelis,default,0.016667,"{'answers': ['John Kukuzelis', 'john kukuzelis...",0.0,0.0,0.0,0.0,0.0,0.666667,0.016667
97,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",Vito Corleone,default,0.000000,"{'answers': ['Vito Andolini', 'vito corleone',...",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
98,[{'content': 'Answer the question based on the...,"[{'content': '<think> First, I need to find ou...",The Île de la Cité,default,0.183333,"{'answers': ['Île de la Cité', 'the île de la ...",0.0,0.0,0.5,0.5,0.0,0.333333,0.183333


In [10]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000
mean,0.190572,0.070000,0.121088,0.37500,0.338667,0.195000,0.453333,0.190572
std,0.200948,0.256432,0.283531,0.37183,0.331026,0.332537,0.362433,0.200948
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.008333,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.008333
50%,0.175000,0.000000,0.000000,0.50000,0.500000,0.000000,0.333333,0.175000
75%,0.276935,0.000000,0.000000,0.50000,0.500000,0.500000,0.666667,0.276935
max,0.754167,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.754167


In [11]:
row = df.iloc[0]